<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/twyncoder/dl-handson-tcb/blob/master/L04_IntroCNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Redes de Aprendizaje Profundo básicas con Keras y Tensorflow.
## *Convolutional Deep Neural Networks (CNN) para clasificación multi-clase*

## 0. Preparación del entorno y comprobación de requisitos

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras

assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
SUBFOLDER = "HandsOn_04"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", SUBFOLDER)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

### Información de versiones

In [2]:
print(tf.__version__)
print(keras.__version__)

2.4.2
2.4.0


### Comprobar si disponemos de una GPU

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 1. Dataset

In [4]:
from sklearn.model_selection import train_test_split
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full,y_train_full,
                                                     train_size=55000,shuffle=True)

### Estandarización de las entradas

In [5]:
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

**¡AHORA TÚ!**
- Observa las dimensiones de `X_mean` y `X_std` y explica cómo se está haciendo la estandarización de los datos de entrada a la red.
- Observa que `X_mean` y `X_shape` se calculan sobre el set de entrenamiento, pero después se aplican también para pseudo estandarizar el set de validación y el set de test, ¿puedes explicar por qué?.

### Ampliación de las dimensiones de entrada
- La entrada a nuestras Redes Neuronales Convolucionales (CNNs) será una imagen de 28x28 píxeles, con un canal de información. Debemos añadir una dimensión a nuestras entradas.
- Comprueba como quedan las dimensiones de los arrays n-dimensionales de _numpy_ con las muestras (entrenamiento, validación y test) después de ejecutar la siguiente celda. Como en ocasiones anteriores, puedes imprimir _shape_.
    - Llamaremos a las dimensiones `(batch_size, height, width, channels)`.

In [8]:
X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

## 2. Entrenamiento

### Modelo 'base' de red neuronal

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=(3, 3),
                 activation='relu', padding='same',
                 kernel_initializer='he_normal',
                 input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), 
                 activation='relu',  padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

**¡AHORA TÚ!**
- Obtén un `summary()` (opcionalmente también un `plot_model()`) de la red anterior y contesta a las siguientes cuestiones:
 - Asegúrate de entender cómo disminuye el tamaño de las capas en `height` y `width` desde 28x28 px a la entrada hasta 7x7 después de la última capa de _pooling_.
 - Busca información sobre `keras.layers.Conv2D()` y averigua qué quiere decir `padding='same'`. ¿Qué otra opción existe para este parámetro y qué implicaciones tiene usarla? Puedes probar a cambiarlo y ver cómo cambian los tamaños de las capas.
 - Describe la red anterior, ¿cuántos _feature maps_ o `channels` tiene la primera capa? ¿de qué tamaño con los _campos receptivos_ o `kernels` de convolución?
 - ¿Cuántos parámetros entrenables tiene la red? Compara esta cifra con el número de parámetros del modelo 'base' de red neuronal _fully connected_ del cuaderno L03.

**¡AHORA TÚ!**
- Entrena la red neuronal durante 15 o 20 epochs con un optimizador `adam`
- Muestra en una gráfica la evolución del entrenamiento.
- Compara los resultados de `accuracy` y `val_accuracy` con los obtenidos con el modelo 'base' de red neuronal _fully connected_ del cuaderno L03.

### Batch Normalization

**¡AHORA TÚ!**
- ¿Se produce _overfitting_? Puedes probar a introducir capas de tipo `Batch Normalization` antes de las capas de _pooling_. Te anticipamos que no mejorará mucho en este caso, pero al menos habrás aprendido a utilizar _batch normalization_ en redes CNN :-)


### L1 and L2 Regularization
**¡AHORA TÚ!**
- También puedes probar a introducir regularización L2.

### Dropout
**¡AHORA TÚ!**
 - Añade capas de dropout después de las capas de _pooling_

### Scheduled learning rate
**¡AHORA TÚ!**
 - Introduce variación dinámica del _learning rate_ con alguna de las soluciones aprendidas

### Aumentando la profundidad de la red

**¡AHORA TÚ!**
- Lo que tenemos hasta ahora no es una red suficientemente _"deep"_. Aumenta el número de capas convolucionales.   
 - Deberías aumentar el número de channels en las capas más profundas de la red, ¿sabes contestar por qué?.
 - Observa que también puedes aumentar el número de neuronas en la capa _hidden_ de perceptrón multicapa (MLP) que hay al final de la red.
 - Controla el número de parámetros entrenables y número de capas en un tamaño manejable para el equipo que estás utilizando.
- Algunas recomendaciones a partir de aquí:
 - No utilices regularización L2 en este experimento, a fin de que el entrenamiento no sea demasiado lento.
 - No obstante, utiliza _Batch normalization_ y _dropout_.
 - Añade además callbacks para variar el _learning rate_ dinámicamente 
 - Recuerda que puedes utilizar _early stopping_ para quedarte con la mejor configuración de pesos de todos los epochs de entrenamiento. Ajusta el parámetro _patience_ dependiendo de la velocidad de tu equipo.
- A ver si puedes obtener un resultado cercano a `val_accuracy` en torno a 93%... ¡o mejor aún!

## 3. Salvar modelo
**¡AHORA TÚ!**
- Salva el modelo entrenado, por si quisieras utilizarlo más adelante

## 4. Test
**¡AHORA TÚ!**
- Carga el modelo salvado anteriormente y evalúalo (usando `evaluate()`), sobre las muestras reservadas para test. 
- Utiliza a continuación el modelo para generar predicciones sobre todo el subconjunto de entrenamiento.

### Matriz de confusión
**¡AHORA TÚ!**
- Utiliza las predicciones anteriores para generar un matriz de confusión normalizada.

### Métricas de rendimiento
**¡AHORA TÚ!**
- Genera ahora una matriz de confusión sin normalizar.
- Calcula la métrica F1-score para cada clase y su valor medio.
- Compara los resultados obtenidos con aquellos que lograste en el cuaderno L03.

## Ampliación

- Blog towardsdatascience: The 4 Convolutional Neural Network Models That Can Classify Your Fashion Images

https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d
- Fashion MNIST benchmark

https://paperswithcode.com/sota/image-classification-on-fashion-mnist